In [ ]:
import sys
sys.path.insert(0, '/mnt/nas/users/mjm/LDPUts')

from discretizer import discretizer
from client import client
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import matplotlib.pyplot as plt
from server import server_ell2, server_multinomial_genrr, server_multinomial_bitflip
from data_generator import data_generator
from discretizer import discretizer
import time
import numpy as np
from scipy.stats import chi2
from utils import chi_sq_dist

In [ ]:
n_test = 200
n_permutation = 199
significance_level = 0.05
alphabet_size = 500
privacy_level = 0.5
###################
sample_size = 400
###################

data_gen = data_generator(device)

p = torch.arange(1,alphabet_size).reciprocal()
p = p.divide(p.sum())

print(p)
plt.plot(p)

In [ ]:
LDPclient = client(device, privacy_level)
server_elltwo = server_ell2(device, privacy_level)
server_genrr = server_multinomial_genrr(device, privacy_level)
server_bitflip = server_multinomial_bitflip(device, privacy_level)

p_value_array = np.zeros([n_test, 3])

t = time.time()
for i in range(n_test):
    print(f"{i+1}th test")
    torch.manual_seed(i)
    data_y = data_gen.generate_multinomial_data(p, sample_size)
    data_z = data_gen.generate_multinomial_data(p, sample_size)
    
    LDPclient.load_data_multinomial(data_y, data_z, alphabet_size)
    data_list_lapu_y, data_list_lapu_z = LDPclient.release_lapu()
    data_list_genrr_y, data_list_genrr_z = LDPclient.release_genrr()
    data_list_bitflip_y, data_list_bitflip_z = LDPclient.release_bitflip()
    
    server_elltwo.load_private_data_multinomial(data_list_lapu_y, data_list_lapu_z, alphabet_size)
    p_value_array[i,0] = server_elltwo.release_p_value_permutation(n_permutation)
 
    server_genrr.load_private_data_multinomial(data_list_genrr_y, data_list_genrr_z, alphabet_size)
    p_value_array[i,1] = server_genrr.release_p_value()
     
    server_bitflip.load_private_data_multinomial(data_list_bitflip_y, data_list_bitflip_z, alphabet_size)
    p_value_array[i,2] = server_bitflip.release_p_value()
elapsed = time.time() - t
print(elapsed)

In [ ]:
print(
   # f"small chi-square distance\n"+
        f"p = {p}\n"+
        f"privacy level = {privacy_level}"
)

print(
    f"sample size = {sample_size},\n"+
    f"type I error estimate =\n"+
    f"    {sum( p_value_array[:,0]<0.05)/n_test }(lapu, ell2)\n"+
    f"    {sum( p_value_array[:,1]<0.05)/n_test }(genrr, chi-sq)\n"+
    f"    {sum( p_value_array[:,2]<0.05)/n_test }(bitflip, ch-sq-proj)\n"
)
with open('/mnt/nas/users/mjm/LDPUts/simul/type_1/hi_dim/power_law/n_400_k_200_alpha_05_type_1_powerlaw.npy', 'wb') as f:
    np.save(f, p_value_array)

In [ ]:
type_1_nominal = np.linspace(0,1,100)
type_1_real_ell2 = np.zeros(100)
type_1_real_genrr = np.zeros(100)
type_1_real_bitflip = np.zeros(100)

for i, level in enumerate(type_1_nominal):
    type_1_real_ell2[i] = sum( p_value_array[:,0]<level)/n_test
    type_1_real_genrr[i] = sum( p_value_array[:,1]<level)/n_test
    type_1_real_bitflip[i] = sum( p_value_array[:,2]<level)/n_test

In [ ]:
plt.plot(type_1_nominal, type_1_nominal)
plt.plot(type_1_nominal, type_1_real_ell2)

In [ ]:
plt.plot(type_1_nominal, type_1_nominal)
plt.plot(type_1_nominal, type_1_real_genrr)

In [ ]:
plt.plot(type_1_nominal, type_1_nominal)
plt.plot(type_1_nominal, type_1_real_bitflip)